In [2]:

#──────────────────────────────────────────────────────────────
# 1. 라이브러리
import pandas as pd, numpy as np, xgboost as xgb
from tqdm import tqdm

#──────────────────────────────────────────────────────────────
# 2. 데이터 로드 및 전처리 (관측 + 예측)
obs  = pd.read_csv('데이터_분석과제_7_기상관측데이터_2401_2503.csv')
pred = pd.read_csv('데이터_분석과제_7_기상예측데이터_2401_2503.csv')
obs.columns  = ['datetime', 'humidity_obs', 'temp_obs', 'pressure_obs']
pred.columns = ['datetime', 'solar_rad_pred', 'humidity_pred', 'absolute_humidity_pred', 'temp_pred', 'pressure_pred']

# 단위 통일 (pressure: mmHg → hPa)
obs['pressure_obs'] = obs['pressure_obs'] * 1.33322
obs['datetime']     = pd.to_datetime(obs['datetime'])
pred['datetime']    = pd.to_datetime(pred['datetime'])

df = pd.merge(obs, pred, on='datetime')

# 오차 변수 생성
df['temp_error']     = df['temp_pred']     - df['temp_obs']
df['humidity_error'] = df['humidity_pred'] - df['humidity_obs']
df['pressure_error'] = df['pressure_pred'] - df['pressure_obs']

# 시간 파생변수
df['hour']  = df['datetime'].dt.hour
df['month'] = df['datetime'].dt.month
df['day']   = df['datetime'].dt.day
df['year']  = df['datetime'].dt.year

df['hour_sin']  = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos']  = np.cos(2 * np.pi * df['hour'] / 24)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

#──────────────────────────────────────────────────────────────
# 3. 이상치 제거
def get_outlier_indices(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return series[(series < lower) | (series > upper)].index

out_idx = set(get_outlier_indices(df['pressure_error']))
out_idx |= set(get_outlier_indices(df['temp_error']))
out_idx |= set(get_outlier_indices(df['humidity_error']))

drop_out = df.drop(index=out_idx).reset_index(drop=True)
ori_data = df.copy()

#──────────────────────────────────────────────────────────────
# 4. 라이브러리
import pandas as pd, numpy as np, itertools, xgboost as xgb
from tqdm import tqdm

#──────────────────────────────────────────────────────────────
# 5. 단일-타깃 지표 (½·RMSE+MAE)
def single_metric(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    mae  = np.mean(np.abs(y_true - y_pred))
    return 0.5 * (rmse + mae)

train = drop_out.query("'2024-01-01' <= datetime < '2025-01-01'")
val   = ori_data .query("'2025-01-01' <= datetime < '2025-03-01'")
test  = ori_data .query("'2025-03-01' <= datetime < '2025-04-01'")

#──────────────────────────────────────────────────────────────
# 6. 피처셋 · 타깃 정의
targets = ["humidity_error", "pressure_error", "temp_error"]

drop_cols = ["datetime","humidity_obs","temp_obs","pressure_obs"] + targets
all_feats = [c for c in ori_data.columns if c not in drop_cols]

sincos = ["hour_sin","hour_cos","month_sin","month_cos"]
feature_sets = {
    "drop_sincos": [c for c in all_feats if c not in sincos],
    "all_features": all_feats
}

# 타깃별 사용할 피처셋
target_fs_map = {
    "humidity_error": "all_features",
    "temp_error"    : "all_features",
    "pressure_error": "drop_sincos"
}

#──────────────────────────────────────────────────────────────
# 7. 파라미터 그리드
param_grid = {
    "max_depth"       : [2,3,4,5, 6,7, 8,9],
    "learning_rate"   : [0.1,0.05,0.02,0.01],
    "subsample"       : [0.4,0.6,0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "n_estimators"    : [100,200, 400],
    "reg_lambda"      : [0, 1,10],
    "reg_alpha"       : [0,0.05,0.1, 0.5],
}
grid_list = list(itertools.product(*param_grid.values()))
print("Grid size:", len(grid_list))

#──────────────────────────────────────────────────────────────
# 8. 학습 & 검증 (타깃별)
models, val_scores, best_params = {}, {}, {}


for tgt, fs_name in target_fs_map.items():
    feats = feature_sets[fs_name]
    X_tr, X_vl = train[feats], val[feats]
    y_tr, y_vl = train[tgt],   val[tgt]

    dtr = xgb.DMatrix(X_tr, label=y_tr)
    dvl = xgb.DMatrix(X_vl, label=y_vl)

    best_score, best_bst, best_p = np.inf, None, None
    print(f"\n🔍 Feature set: {fs_name} | Target: {tgt}")
    for combo in tqdm(grid_list, leave=False):
        params = dict(zip(param_grid.keys(), combo),
                      tree_method = "gpu_hist",
                      predictor   = "gpu_predictor",
                      verbosity   = 0)
        bst   = xgb.train(params, dtr, num_boost_round=params["n_estimators"])
        score = single_metric(y_vl.values, bst.predict(dvl))
        if score < best_score:
            best_score, best_bst, best_p = score, bst, params

    models[(fs_name, tgt)]  = best_bst
    val_scores[(fs_name,tgt)] = best_score
    best_params[tgt]        = {k:v for k,v in best_p.items() if k in param_grid}
    print(f"✅ Best Val sAError = {best_score:.4f}")

#──────────────────────────────────────────────────────────────
# 9. Test 평가
preds = test[["datetime"]].copy()

for tgt, fs_name in target_fs_map.items():
    feats = feature_sets[fs_name]
    dtest = xgb.DMatrix(test[feats])
    pred  = models[(fs_name, tgt)].predict(dtest)

    score = single_metric(test[tgt].values, pred)
    print(f"🎯 Feature set: {fs_name:<11s}| {tgt:<16s} Test sAError = {score:.6f}")

    preds[f"{tgt}_pred"] = pred


#──────────────────────────────────────────────────────────────
# 10. 타깃별 최적 하이퍼파라미터 출력
print("\n🛠️  Best hyperparameters per target")
for tgt, p in best_params.items():
    print(f"\n[{tgt}]")
    for k, v in p.items():
        print(f"  {k:<17s}: {v}")


Grid size: 9216

🔍 Feature set: all_features | Target: humidity_error


KeyboardInterrupt: 